In [181]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 30)
types = {'id': int, 'ciudad':str, 'provincia': str}

training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','provincia','ciudad','idzona','metrostotales','precio'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv',dtype = types, usecols=['id','provincia','ciudad','idzona','lat','lng'])

In [182]:
training_data.precio.mean()/training_data.metrostotales.mean()

14317.519170109566

In [183]:
metros_ciudad = training_data.groupby('ciudad').agg({'metrostotales':'mean'}).reset_index().rename(columns={'metrostotales':'mts_ciudad'})

meanmts=metros_ciudad.mts_ciudad.mean()
metros_ciudad.mts_ciudad.fillna(meanmts,inplace=True)

training_data = pd.merge(training_data,metros_ciudad, on='ciudad', how='right')

In [184]:
training_data.metrostotales.fillna(training_data.mts_ciudad,inplace=True)
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239628 entries, 0 to 239627
Data columns (total 7 columns):
id               239628 non-null int32
ciudad           239628 non-null object
provincia        239628 non-null object
metrostotales    239628 non-null float64
idzona           211379 non-null float64
precio           239628 non-null float64
mts_ciudad       239628 non-null float64
dtypes: float64(4), int32(1), object(2)
memory usage: 13.7+ MB


In [185]:
training_data.idzona.fillna(-1, inplace=True)
props_zone = ((training_data.idzona.value_counts()>5).to_frame().reset_index()).rename(columns={'index':'idzona','idzona':'+5props'})
props_zone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19720 entries, 0 to 19719
Data columns (total 2 columns):
idzona     19720 non-null float64
+5props    19720 non-null bool
dtypes: bool(1), float64(1)
memory usage: 173.4 KB


In [186]:
training_data.ciudad.fillna('no_city', inplace=True)

In [187]:
precio_ciudad = training_data.groupby('ciudad').agg({'precio':'sum','metrostotales':'sum'}).reset_index()
precio_ciudad['precio_mts_ciudad'] = precio_ciudad.precio/precio_ciudad.metrostotales
precio_ciudad.drop(columns={'metrostotales','precio'},inplace=True)
precio_ciudad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 2 columns):
ciudad               875 non-null object
precio_mts_ciudad    875 non-null float64
dtypes: float64(1), object(1)
memory usage: 13.8+ KB


In [188]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239628 entries, 0 to 239627
Data columns (total 7 columns):
id               239628 non-null int32
ciudad           239628 non-null object
provincia        239628 non-null object
metrostotales    239628 non-null float64
idzona           239628 non-null float64
precio           239628 non-null float64
mts_ciudad       239628 non-null float64
dtypes: float64(4), int32(1), object(2)
memory usage: 13.7+ MB


In [189]:
training_data = pd.merge(training_data,props_zone, on='idzona', how='right')
training_data = pd.merge(training_data,precio_ciudad, on= 'ciudad', how='right')
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239628 entries, 0 to 239627
Data columns (total 9 columns):
id                   239628 non-null int32
ciudad               239628 non-null object
provincia            239628 non-null object
metrostotales        239628 non-null float64
idzona               239628 non-null float64
precio               239628 non-null float64
mts_ciudad           239628 non-null float64
+5props              239628 non-null bool
precio_mts_ciudad    239628 non-null float64
dtypes: bool(1), float64(5), int32(1), object(2)
memory usage: 15.8+ MB


In [190]:
training_data.tail(100)

,id,ciudad,provincia,metrostotales,idzona,precio,mts_ciudad,+5props,precio_mts_ciudad
239528,126166,Elota,Sinaloa,195.646962,93794.0,2900000.0,195.646962,False,14822.617065
239529,26760,Ciudad Fernández,San luis Potosí,105.000000,90540.0,2000000.0,105.000000,False,14285.714286
239530,154463,Ciudad Fernández,San luis Potosí,105.000000,90557.0,1000000.0,105.000000,False,14285.714286
239531,289783,General Plutarco Elías Calles,Sonora,300.000000,94948.0,1300000.0,300.000000,False,4333.333333
239532,197575,Huatusco,Veracruz,105.000000,110204.0,1590000.0,105.000000,False,15142.857143
239533,174533,Mascota,Jalisco,231.000000,49746.0,1100000.0,175.500000,False,10000.000000
239534,293265,Mascota,Jalisco,175.500000,49746.0,1390000.0,175.500000,False,10000.000000
239535,225097,Mascota,Jalisco,175.500000,49746.0,4100000.0,175.500000,False,10000.000000
239536,273260,Mascota,Jalisco,120.000000,49746.0,430000.0,175.500000,False,10000.000000
239537,232116,Sabinas Hidalgo,Nuevo León,120.000000,69376.0,2600000.0,120.000000,False,12937.083333


In [191]:
precio_zona = training_data.groupby('idzona').agg({'precio':'sum','metrostotales':'sum','+5props':'median','precio_mts_ciudad':'mean'}).reset_index()
precio_zona.loc[precio_zona['+5props'], 'precio_por_metro'] = precio_zona.precio/precio_zona.metrostotales
precio_zona.loc[~(precio_zona['+5props']), 'precio_por_metro'] = precio_zona.precio_mts_ciudad
precio_zona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19720 entries, 0 to 19719
Data columns (total 6 columns):
idzona               19720 non-null float64
precio               19720 non-null float64
metrostotales        19720 non-null float64
+5props              19720 non-null bool
precio_mts_ciudad    19720 non-null float64
precio_por_metro     19720 non-null float64
dtypes: bool(1), float64(5)
memory usage: 789.6 KB


In [192]:
precio_zona = precio_zona[['idzona','precio_por_metro']]

In [194]:
precio_zona.head()

,idzona,precio_por_metro
0,-1.0,12110.166717
1,22.0,25368.514320
2,23.0,8060.561494
3,26.0,5430.679406
4,27.0,8758.432621


In [195]:
precio_zona.to_csv('../../res/ftr/precio_por_metro_idzona.csv')